# 使用 SQLite 儲存記憶體

## 概覽

本教學說明了 ```SQLChatMessageHistory``` 類別的使用方式，該類別允許將聊天記錄儲存至任何由 ```SQLAlchemy``` 支援的資料庫中。

```SQL（結構化查詢語言）``` 是一種領域專用語言，用於程式設計中管理關聯式資料庫管理系統（RDBMS）中的資料，或用於關聯式資料流管理系統（RDSMS）中的資料流處理。它特別適合處理結構化資料，包括實體與變數間的關係。

```SQLAlchemy``` 是一個開源的 **SQL** 工具包與物件關聯對應（ORM）框架，適用於 Python 程式語言，採用 MIT 授權釋出。

若你希望使用非 ```SQLite``` 的資料庫，請先確保已安裝該資料庫對應的驅動程式。

### 目錄

- [概覽](#overview)
- [環境設置](#environment-setup)
- [使用方式](#Usage)
- [鏈結應用](#Chaining)

### 參考資料

- [Wikipedia：SQL](https://en.wikipedia.org/wiki/SQL)
- [SQLAlchemy 官方 GitHub](https://github.com/sqlalchemy/sqlalchemy)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain_opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain_community",
        "langchain_openai",
        "langchain_core",
        "SQLAlchemy",
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "MemoryUsingSQLite",
    }
)

Environment variables have been set successfully.


You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it.

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [ ]:
# from dotenv import load_dotenv

# load_dotenv()

True

## 使用方式

要使用此儲存機制，你只需提供以下兩個項目：

1. ```session_id``` - 一個唯一的會話識別碼，例如使用者名稱、電子郵件、聊天 ID 等。

2. ```connection``` - 指定資料庫連線的字串。此字串將傳遞給 SQLAlchemy 的 ```create_engine``` 函式。

In [6]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Initialize chat history with session ID and database connection.
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

In [7]:
# Add a user message
chat_message_history.add_user_message(
    "Hello, nice to meet you! My name is Heesun :) I'm a LangChain developer. I look forward to working with you!"
)
# Add an AI message
chat_message_history.add_ai_message(
    "Hi, Heesun! Nice to meet you. I look forward to working with you too!"
)

Now, let's check the stored conversation history.

In [9]:
chat_message_history.messages

[HumanMessage(content="Hello, nice to meet you! My name is Heesun :) I'm a LangChain developer. I look forward to working with you!", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi, Heesun! Nice to meet you. I look forward to working with you too!', additional_kwargs={}, response_metadata={})]

You can also clear the session memory from db:

In [10]:
# Clear the session memory
chat_message_history.clear()
chat_message_history.messages

[]

### 新增中繼資料（Metadata）

你可以透過直接建立 ```HumanMessage``` 與 ```AIMessage``` 物件來新增 **中繼資料（Metadata）**。這種方式可提供更靈活的資料處理與記錄功能。

**參數說明**：
- ```additional_kwargs``` - 儲存自定標籤或中繼資料，例如優先順序或任務類型。

- ```response_metadata``` - 捕捉 AI 回應的詳細資訊，包括使用的模型、時間戳記與 token 數量。

這些欄位可透過詳細的資料儲存方式，提升除錯與任務追蹤的能力。

In [11]:
from langchain_core.messages import HumanMessage

# Add a user message with additional metadata.
user_message = HumanMessage(
    content="Can you help me summarize this text?",
    additional_kwargs={"task": "summarization"},
)

# Add the message to chat history.
chat_message_history.add_message(user_message)

In [12]:
chat_message_history.messages

[HumanMessage(content='Can you help me summarize this text?', additional_kwargs={'task': 'summarization'}, response_metadata={})]

In [13]:
from langchain_core.messages import AIMessage

# Add an AI message with response metadata.
ai_message = AIMessage(
    content="Sure! Here's the summary of the provided text.",
    response_metadata={"model": "gpt-4", "token_count": 30, "response_time": "150ms"},
)

# Add the message to chat history.
chat_message_history.add_message(ai_message)

In [14]:
chat_message_history.messages

[HumanMessage(content='Can you help me summarize this text?', additional_kwargs={'task': 'summarization'}, response_metadata={}),
 AIMessage(content="Sure! Here's the summary of the provided text.", additional_kwargs={}, response_metadata={'model': 'gpt-4', 'token_count': 30, 'response_time': '150ms'})]

## Chaining

You can easily integrate this chat history class with [LCEL Runnables](https://wikidocs.net/235884).

In [15]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        # Placeholder for chat history
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

# Chaining
chain = prompt | ChatOpenAI(model_name="gpt-4o") | StrOutputParser()

The following shows how to create a function that returns chat history from ```sqlite.db```.

In [17]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

Set ```config_fields``` to provide reference information when retrieving conversation details.

In [18]:
"""
這段程式碼的目的是定義兩個可配置的欄位：user_id 和 conversation_id。這些欄位可以讓 LangChain 的應用程式或執行鏈在運行時接收這些特定的 ID，並且由於 is_shared=True，這些 ID 可以在應用程式的不同組件之間共享，例如用於追蹤特定用戶的對話歷史或上下文。

這通常用於需要根據不同用戶或不同對話情境來調整行為的應用程式，例如在聊天機器人中，用來區分不同的使用者對話，以便正確地存取和管理各自的記憶或狀態。
"""
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [19]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="question",
    history_messages_key="chat_history",
    # Set parameters for retrieving chat history
    history_factory_config=config_fields,
)

Set the ```"user_id"``` and ```"conversation_id"``` key-value pairs under the ```"configurable"``` key.

In [20]:
# Config settings
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

Let's ask a question about the name. If there is any previously saved conversation history, it will provide the correct response.  

- Use the ```invoke``` method of the ```chain_with_history``` object to generate an answer to the question.  
- Pass a question dictionary and ```config``` settings to the ```invoke``` method as inputs.  

In [21]:
# Execute by passing the question and config
chain_with_history.invoke(
    {"question": "Hi, nice to meet you. My name is Heesun."}, config
)

'Hello, Heesun! Nice to meet you too. How can I assist you today?'

In [94]:
# Execute a follow-up question
chain_with_history.invoke({"question": "What is my name?"}, config)

'Your name is Heesun.'

This time, set the same ```user_id``` but use a different value for ```conversation_id```.

In [95]:
# Config settings
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# Execute by passing the question and config
chain_with_history.invoke({"question": "What is my name?"}, config)

"I'm sorry, but I don't have access to personal information, so I don't know your name. If you'd like, you can tell me your name, and I can address you by it."